In [ ]:
# 1. Import required libraries
import pandas as pd
from sklearn.linear_model import LogisticRegression



In [ ]:
from sklearn.preprocessing import LabelEncoder ,StandardScaler

# Load the dataset
df_final = pd.read_csv('Titanic-Dataset.csv')

# Drop columns that are not useful for modeling
df_final = df_final.drop(columns =['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Handle missing values
df_final['Age'] = df_final['Age']. fillna(df_final['Age'].median ())
df_final['Embarked'] = df_final['Embarked']. fillna(df_final['Embarked'].mode ()[0])

# Encode 'Sex' using LabelEncoder
le = LabelEncoder ()
df_final['Sex'] = le.fit_transform(df_final['Sex'])

# One -hot encode 'Embarked '
df_final = pd.get_dummies(df_final , columns =['Embarked'],drop_first=True)

# Standardize numerical features
scaler = StandardScaler ()
df_final [['Age', 'Fare']] = scaler.fit_transform(df_final [['Age', 'Fare']])

This code performs preprocessing on the **Titanic dataset** to prepare it for machine learning. First, it drops irrelevant columns (`PassengerId`, `Name`, `Ticket`, and `Cabin`) that don't contribute meaningful information for predicting survival. It then fills missing values in the `Age` column with the median and in the `Embarked` column with the most frequent value (mode). The `Sex` column, which is categorical, is encoded into numeric values using `LabelEncoder` (e.g., male as 1, female as 0). The `Embarked` column is one-hot encoded, creating separate binary columns for each port, with one dropped to avoid redundancy. Finally, the `Age` and `Fare` features are standardized using `StandardScaler`, so they have a mean of 0 and a standard deviation of 1, which helps many machine learning models perform better. The result is a clean, fully numeric, and scaled DataFrame ready for training a model.

In [ ]:
# Split Data for Training and Testing
from sklearn.model_selection import train_test_split
X = df_final.drop('Survived', axis =1) # Features
y = df_final['Survived'] # Target
X_train , X_test , y_train , y_test = train_test_split(X, y,test_size =0.2, random_state =42)

This code splits the preprocessed Titanic dataset into **training and testing sets**, which is an essential step before fitting a machine learning model.

- `X` contains the **features** (all columns except `Survived`), which are the inputs used to predict the outcome.
- `y` contains the **target variable**, `Survived`, which indicates whether a passenger survived (1) or not (0).
- `train_test_split()` splits the data into:
  - `X_train`, `y_train`: used to **train** the model (80% of the data)
  - `X_test`, `y_test`: used to **evaluate** the model’s performance (20% of the data)

The `random_state=42` ensures reproducibility—every time you run the code, you get the same split.

This prepares your data for training a model like logistic regression, decision trees, or any other supervised learning algorithm.

In [ ]:
df = pd.read_csv('Titanic-Dataset.csv')

lasso_model = LogisticRegression(penalty='l1', solver='liblinear')
# 'liblinear ' supports L1
lasso_model.fit(X_train ,y_train)
lasso_coeffs = pd.DataFrame ({'Feature': X.columns ,'Coefficient': lasso_model.coef_ [0]})
lasso_coeffs.sort_values(by='Coefficient', ascending=False)

Feature  Coefficient
5        Fare     0.137554
6  Embarked_Q     0.000000
4       Parch    -0.095114
3       SibSp    -0.290685
7  Embarked_S    -0.300348
2         Age    -0.373984
0      Pclass    -0.901768
1         Sex    -2.607504

This code applies Lasso (L1) regularized logistic regression to the Titanic dataset in order to identify the most important features for predicting passenger survival. After loading the dataset (though this step is redundant if preprocessing was already done), it creates a logistic regression model with an L1 penalty using the `liblinear` solver, which supports this form of regularization. The model is trained on the training data (`X_train`, `y_train`), and the resulting feature coefficients are extracted into a DataFrame. These coefficients indicate the strength and direction of each feature's influence on survival. By sorting the coefficients, we can see which features are most significant in the prediction, while less important ones may have coefficients close to zero or exactly zero—effectively removing them from the model. This helps in both simplifying the model and highlighting the key factors that impact survival.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Initialize RFE with Logistic Regression as the estimator
rfe = RFE(estimator=LogisticRegression(solver='liblinear'), n_features_to_select=5)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Extract the selected features
selected_features = X.columns[rfe.support_]

# Display the result
print("Selected by RFE:", list(selected_features))


Selected by RFE: ['Pclass', 'Sex', 'Age', 'SibSp', 'Embarked_S']


This code demonstrates the use of Recursive Feature Elimination (RFE) for feature selection in machine learning. RFE is a technique that recursively removes the least important features and builds a model on the remaining features, ranking them by their importance. In this case, the estimator used is Logistic Regression with the 'liblinear' solver. The code first initializes the RFE object, specifying that five features should be selected. It then fits the RFE model on the training data (`X_train` and `y_train`), where `X_train` contains the input features and `y_train` contains the target variable. After fitting, the `support_` attribute of the RFE object identifies which features are selected (True or False for each feature), and these are extracted using `X.columns`. Finally, the selected features are printed.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Select final features
selected_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Embarked_S']
X_selected = df_final[selected_features]
y = df_final['Survived']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the logistic regression model
model = LogisticRegression(solver='liblinear')  # Added solver argument
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate using confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[89 16]
 [21 53]]

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       105
           1       0.77      0.72      0.74        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



This code trains a logistic regression model to predict the survival of Titanic passengers based on selected features. First, it selects relevant features (such as 'Pclass', 'Sex', 'Age', 'SibSp', and 'Embarked_S') from the dataset. The data is then split into training and testing sets using `train_test_split`, with 20% of the data reserved for testing. The logistic regression model is trained on the training data (`X_train`, `y_train`), and predictions are made on the test data (`X_test`). The performance of the model is evaluated by printing the confusion matrix, which shows the number of correct and incorrect predictions, and the classification report, which provides detailed metrics such as precision, recall, and F1-score for each class (survived or not).

In [ ]:
# Predict survival probabilities
y_probs = model.predict_proba(X_selected)[:, 1]

# Create 3-class survival labels based on custom thresholds
import numpy as np
survival_class = np.select([y_probs < 0.4, (y_probs >= 0.4) & (y_probs < 0.7), y_probs >= 0.7], [0, 1, 2])

# Check the distribution of the classes
from collections import Counter
print("Class Distribution:", Counter(survival_class))

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, survival_class, test_size=0.2, random_state=42
)

# Train multinomial logistic regression model
from sklearn.linear_model import LogisticRegression
multi_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
multi_model.fit(X_train, y_train)

# Train One-vs-Rest logistic regression model
ovr_model = LogisticRegression(multi_class='ovr', solver='liblinear')
ovr_model.fit(X_train, y_train)

# Evaluate both models
from sklearn.metrics import classification_report
print("Multinomial Logistic Regression Model Evaluation:\n", classification_report(y_test, multi_model.predict(X_test)))
print("One-vs-Rest Logistic Regression Model Evaluation:\n", classification_report(y_test, ovr_model.predict(X_test)))


Class Distribution: Counter({np.int64(0): 528, np.int64(1): 185, np.int64(2): 178})
Multinomial Logistic Regression Model Evaluation:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98        97
           1       0.91      0.93      0.92        42
           2       0.97      0.93      0.95        40

    accuracy                           0.96       179
   macro avg       0.95      0.95      0.95       179
weighted avg       0.96      0.96      0.96       179

One-vs-Rest Logistic Regression Model Evaluation:
               precision    recall  f1-score   support

           0       0.87      1.00      0.93        97
           1       0.88      0.55      0.68        42
           2       0.90      0.93      0.91        40

    accuracy                           0.88       179
   macro avg       0.88      0.82      0.84       179
weighted avg       0.88      0.88      0.87       179



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


This code predicts the survival probabilities of Titanic passengers using a logistic regression model and then creates three survival classes based on custom probability thresholds (less than 0.4 for class 0, between 0.4 and 0.7 for class 1, and greater than or equal to 0.7 for class 2). It then splits the data into training and testing sets and trains two different logistic regression models: a multinomial logistic regression and a one-vs-rest (OvR) logistic regression. After training, both models are evaluated on the test set using the classification report, which provides metrics such as precision, recall, and F1-score for each class. The class distribution of the survival labels is also displayed for reference.

In [ ]:
import numpy as np
import pandas as pd

# Use the model you want to analyze
# For example: model = trained binary logistic regression
# Or: multi_model = trained multinomial logistic regression

# Get coefficients for binary logistic regression or first class in multinomial
coefficients = model.coef_[0]  # Adjust based on the model type (e.g., binary or multiclass)

# Create a DataFrame to show the Feature, Coefficient, and Odds Ratio
coef_summary = pd.DataFrame({
    'Feature': X_selected.columns,
    'Coefficient': coefficients,
    'Odds Ratio': np.exp(coefficients)
})

# Sort the DataFrame by the absolute value of the coefficients to highlight the most impactful features
coef_summary = coef_summary.loc[coef_summary['Coefficient'].abs().sort_values(ascending=False).index]

# Display the result
print(coef_summary)


      Feature  Coefficient  Odds Ratio
1         Sex    -2.438310    0.087308
0      Pclass    -0.896237    0.408103
2         Age    -0.351338    0.703746
4  Embarked_S    -0.318381    0.727326
3       SibSp    -0.273327    0.760844


This code extracts the coefficients from a trained logistic regression model (binary or multinomial) and calculates the corresponding odds ratios for each feature. It creates a DataFrame displaying the features, their coefficients, and the odds ratios (calculated as the exponent of the coefficients). The DataFrame is then sorted by the absolute value of the coefficients to highlight the features with the most significant impact on the model's predictions. Finally, it prints the sorted table, which helps to understand which features contribute most to the model's decision-making process.
